# Исследование прогнозирования успеха стартапов с использованием методов науки о данных

## Введение

В современном мире стартапы играют ключевую роль в экономическом развитии, но лишь немногие из них достигают успеха. Для инвесторов и предпринимателей важно понимать, какие факторы влияют на успешность стартапа. В данном исследовании мы обращаемся к методам анализа данных для построения моделей, способных прогнозировать исход предпринимательских начинаний.

## Цель

Целью исследования является применение методов анализа данных для изучения факторов, влияющих на успешность стартапов, а также для создания моделей, способных предсказывать их результаты.

## План работы

- **Загрузка и ознакомление с данными**: загрузка и изучение наборов данных о стартапах, включая тренировочные и тестовые выборки.

- **Предварительная обработка данных**: очистка и подготовка данных для дальнейшего анализа, включая обработку пропущенных значений и выбросов.

- **Разведочный анализ**: анализ данных с целью выявить закономерности и тренды, а также для поиска ключевых признаков, влияющих на успешность стартапов.

- **Создание новых признаков**: разработка новых признаки на основе имеющихся данных для повышения качества моделей.

- **Обучение моделей**: выбор и обучение модели машинного обучения для прогнозирования успешности стартапов.

- **Оценка качества моделей**: оценка качества построенных моделей с использованием подходящих метрик и методов валидации.

- **Анализ важности признаков**: анализ важности признаков для выявления ключевых факторов, влияющих на результаты предпринимательской деятельности.

---

# Подготовка среды

In [ ]:
import os
import requests

In [ ]:
url = 'https://raw.githubusercontent.com/Lighter01/startups_categories/main/env.yml'

yaml_file = './datasets/env.yml'

if os.path.exists(yaml_file):
    print("File already exists.")
else:
    response = requests.get(url)

    if response.status_code == 200:
        with open(yaml_file, 'wb') as json_file:
            yaml_file.write(response.content)
        print("YAML file with environment dependencies downloaded successfully.")
    else:
        print("Failed to download YAML file.")

In [ ]:
yaml_file = './datasets/env.yml'

os.system(f'conda install --file {yaml_file}')

print("Packages installed successfully into the current environment.")

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
# import umap

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, TargetEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, StratifiedKFold, StratifiedShuffleSplit, RepeatedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.validation import check_is_fitted

import shap

from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline

import optuna
from optuna.trial import Trial
import miceforest as mf
from miceforest import mean_match_default
# from optuna.samplers.nsgaii import VSBXCrossover

import xgboost as xgb
from catboost import CatBoostClassifier, metrics, Pool, cv
import lightgbm as lgb

# from typing import List, Tuple, Dict, Any, Callable, Union
# from pandas import DataFrame, Series

import phik
from phik.report import plot_correlation_matrix

import sys
import time
from collections import defaultdict, Counter


import json
from fuzzywuzzy import fuzz, process
# import us
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup

In [ ]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.precision", 3)

RANDOM_STATE = 12345

In [ ]:
palette = px.colors.qualitative.Plotly

fig = go.Figure()
fig.add_trace(go.Bar(x=[i for i in range(0, len(palette))], y=[1] * len(palette), marker_color=palette))

fig.update_layout(
    width=800,
    height=400,
    title="Color palette",
)

fig.show()

In [ ]:
path1 = './datasets/kaggle_startups_train_01.csv'
path2 = './datasets/kaggle_startups_test_01.csv'
path3 = './datasets/kaggle_startups_sample_submit_01.csv'

def read_file(path):
    if os.path.exists(path):
        df = pd.read_csv(path, sep=',')
    else:
        print('No such file or directory')
    return df

df_train   = read_file(path1)
df_test    = read_file(path2)
df_sampsub = read_file(path3)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_sampsub.head()

---

---

# IDA / preprocessing

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_sampsub.info()

In [ ]:
def display_nans(df):
    nans_per_col = [(col, df[col].isna().sum(), df[col].isna().sum() / df.shape[0] * 100) for col in df.columns]
    dtype = [('col_name', 'U20'), ('nans', int), ('nans_perc', float)]
    nans_per_col = np.array(nans_per_col, dtype=dtype)
    nans_per_col = nans_per_col[nans_per_col['nans'] > 0]
    nans_per_col = np.sort(nans_per_col, order='nans')

    df_show = pd.DataFrame(nans_per_col[::-1])
    display(df_show.style.background_gradient(cmap='Blues'))
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))

    y_pos = np.arange(len(nans_per_col))
    
    ax.barh(y_pos, nans_per_col['nans_perc'], alpha=0.8, edgecolor='black', linewidth=1) 
    ax.set_yticks(y_pos, labels=nans_per_col['col_name'])
    ax.set_xlabel('Nans, %', fontsize=14)
    ax.set_title('Nans rate for each column', fontsize=16)
    ax.set_xlim(0, 100)
    ax.tick_params(axis='both', which='major', labelsize=11)
    ax.grid(axis='x', linestyle='--', linewidth=0.5)
    
    plt.show()

In [ ]:
display_nans(df_train)

In [ ]:
display_nans(df_test)

Проверка на полностью пустые строки

In [ ]:
print(df_train.isna().all(axis=1).sum(), df_test.isna().all(axis=1).sum())

Проверка на явные дубликаты

In [ ]:
print(df_train.duplicated(keep=False).sum())

### name

In [ ]:
df_train = df_train.drop(columns=['name'])
df_test  = df_test.drop(columns=['name'])
df_train.shape, df_test.shape

### founded_at, first_funding_at, last_funding_at, closed_at

Приведем все признаки, содержащие даты, к типу datetime.

In [ ]:
def cast_to_datetime(df, columns):
    for column in columns:
        df[column] = pd.to_datetime(df[column], errors='coerce', format='%Y-%m-%d')
    return df

df_train = cast_to_datetime(df_train, ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])
df_test  = cast_to_datetime(df_test,  ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])
df_train.dtypes

Проверим даты на соответствие реальности.

In [ ]:
def plot_date_range(df, columns):
    fig = make_subplots(rows=2, cols=2, subplot_titles=columns)
    
    for i, column in enumerate(columns, start=1):
        row = 1 if i <= 2 else 2
        col = i if i <= 2 else i - 2
        trace = go.Box(y=df[column].dt.year, name='', showlegend=False)
        fig.add_trace(trace, row=row, col=col)
    
    fig.update_layout(
        height=800,
        width=1000,
        title='Boxplots of dates by Year'
    )
    
    fig.show()

plot_date_range(df_train, ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])

In [ ]:
plot_date_range(df_test, ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])

In [ ]:
def print_date_range(df, cols):
    for col in cols:
        print(col, df[col].min(), df[col].max())

print_date_range(df_train, ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])
print()
print_date_range(df_test,  ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])

Имеются явно аномальные даты из будущего в тренировочной выборке, от которых мы просто избавимся, т.к. их немного. Со старыми стартапами сложнее, т.к. сложно выбрать правильную точку отсчета начала существования старапов. Возьмем в качестве приблизительного начала отсчета 1970 год. Все записи, содержащие даты до этого года не включительно, в тренировочной выборке будут удалены. В тестовой выборке все даты, не соответствующие нижней границе, будут приведены к нижней границе.

In [ ]:
def drop_rows_date(df, cols):
    for col in cols:
        greater = df[df[col] >= pd.Timestamp('2018-01-01')].index
        lower   = df[df[col] <  pd.Timestamp('1970-01-01')].index
        print(f'Startups before 1970 in {col} column: {lower.shape[0]}')
        print(f'Startups after  2017 in {col} column: {greater.shape[0]}')
        df = df.drop(index = greater)
        df = df.drop(index = lower)
    return df

df_train = drop_rows_date(df_train, ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])
print(df_train.shape)

In [ ]:
for dt_col in ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at']:
    df_test[dt_col] = df_test[dt_col].clip(lower='1970-01-01')
    print(dt_col, df_test[dt_col].min())

Проверим на наличие в выборке стартапов, получивших свое первое финансирование еще до своего основания.

In [ ]:
df_train[df_train['founded_at'].dt.year > df_train['first_funding_at'].dt.year].shape

In [ ]:
df_test[df_test['founded_at'].dt.year > df_test['first_funding_at'].dt.year].shape

В датасетах имеются записи о стартапах, у которых первые раунды финансирования прошли до основания стартапа. Не уверен, возможно ли такое. Теоретически, можно предположить, что речь идет о получении финансирования до официального оформления бизнеса. В такой формулировке подобные преценденты вполне логичны, однако существует ли такая практика на самом деле - в этом есть некоторые сомнения. Однако на данный момент оставим подобные записи в выборке, в дальнейшем выделив новый признак, который выделит подобные записи среди общей массы.

In [ ]:
mask_1 = (df_train['closed_at'].notna()) & (df_train['status'] == 'operating')
mask_2 = (df_train['closed_at'].isna())  & (df_train['status'] == 'closed')
print(mask_1.sum(), mask_2.sum())

В выборке нет несоответствий даты закрытия стартапа и статуса.

In [ ]:
del mask_1, mask_2

*

Т.к. в тренировочной выборке всего 21 запись с пропущенной датой первого раунда финансирования, присвоим таким записям в данное поле дату основания стартапа.

In [ ]:
mask = df_train['first_funding_at'].isna()
df_train.loc[mask, 'first_funding_at'] = df_train[mask]['founded_at']
df_train['first_funding_at'].isna().sum()

In [ ]:
mask = df_train['last_funding_at'] - df_train['first_funding_at'] < pd.Timedelta(0)
df_train[mask]

Удалим строку, в которой дата последнего раунда финансирования раньше первого раунда

In [ ]:
df_train = df_train[~mask]
df_train.shape

### funding_rounds

In [ ]:
# def plot_cat_distr(df, col, title='', color=palette[0]):
#     SLICE = 15
#     df_category = df[col].value_counts()
#     display(df_category.to_frame()[:SLICE].T)
    
#     sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})
    
#     df_category[:SLICE].plot(kind='bar', stacked=True, figsize=(10, 6), width=0.8, color=color)
    
#     plt.xlabel('Категория', fontsize=12)
#     plt.ylabel('Число стартапов', fontsize=12)
#     plt.title(title, fontsize=14)
#     plt.xticks(rotation=45)
#     plt.show()

In [ ]:
def show_col_distr_cat(df, col, slice=None, sort_index=False, sort_value=True,
                       color=palette[0], title='', x_axis_t='', y_axis_t='', show=True, stacking_col=''):
    fig = go.Figure()

    if stacking_col != '':
        pivot_table = df.pivot_table(index=col, columns=stacking_col, aggfunc='size', fill_value=0)
        pivot_table.columns = ['closed', 'opened']
        if sort_value:
            pivot_table = pivot_table.sort_values(by=['opened', 'closed'], ascending=False)
        if slice != None:
            pivot_table = pivot_table[:slice]
        display(pivot_table.T)

        order = []
        for i, cat in enumerate(df[stacking_col].unique()):
            local_distr = df[df[stacking_col] == cat][col].value_counts()
            if sort_index:
                local_distr = local_distr.sort_index()
            if len(order) == 0:
                if slice != None:
                    local_distr = local_distr[:slice]
                order = local_distr.index
            else:
                local_distr = local_distr[pd.Index(set.intersection(set(order), set(local_distr.index)))]
                
            local_trace = go.Bar(x=local_distr.index, y=local_distr.values, name=cat, marker_color=palette[i], showlegend=True, opacity=0.8)
            fig.add_trace(local_trace)

        fig.update_layout(
            barmode='stack',
            height=600,
            width=1000,
            title=title,
            xaxis_title=x_axis_t,
            yaxis_title=y_axis_t,
            xaxis=dict(
                dtick=1
            )
        )

        if show:
            fig.show()
            
        return fig
    else:
        distr = df[col].value_counts()
        if sort_index:
            distr = distr.sort_index()
            
        if slice != None:
            distr = distr[:slice]
            
        display(distr.to_frame().T.style.set_caption(' '.join(title.split(' ')[-2:])))
        
        trace = go.Bar(x=distr.index, y=distr.values, name='', marker_color=color, showlegend=False, opacity=0.8)
        fig.add_trace(trace)
    
        fig.update_layout(
            height=600,
            width=1000,
            title=title,
            xaxis_title=x_axis_t,
            yaxis_title=y_axis_t,
            xaxis=dict(
                dtick=1
            )
        )

        if show:
            fig.show()

    return trace

In [ ]:
train_fig = show_col_distr_cat(df_train, 'funding_rounds', sort_index=True, title='Funding rounds distribution for train set', stacking_col='status')
test_tr   = show_col_distr_cat(df_test,  'funding_rounds', sort_index=True, title='Funding rounds distribution for test set', color=palette[2])

In [ ]:
df_train[df_train['funding_rounds'] > 8]

In [ ]:
mask = (df_train['first_funding_at'] == df_train['last_funding_at']) & (df_train['funding_rounds'] > 1)
mask.sum()

В выборке имеются записи, в которых число раундов финансирования больше 1, но при этом даты первого и последнего раунда совпадают. Заменим в таких строках число раундов на единицу. 

In [ ]:
df_train.loc[mask, 'funding_rounds'] = 1

In [ ]:
mask = (df_test['first_funding_at'] == df_test['last_funding_at']) & (df_test['funding_rounds'] > 1)
print(mask.sum())
df_test.loc[mask, 'funding_rounds'] = 1

### funding_total_usd

In [ ]:
def plot_boxplot(df, col, color=palette[0], title='', show=True):
    fig = go.Figure()
    
    trace = go.Box(y=df[col], name='', marker_color=color, showlegend=False)
    fig.add_trace(trace)
    
    fig.update_layout(
        height=600,
        width=400,
        title=title
    )

    if show:
        fig.show()

    return trace

In [ ]:
fund_total_train_tr = plot_boxplot(df_train, 'funding_total_usd', show=False)
fund_total_test_tr  = plot_boxplot(df_test,  'funding_total_usd', color=palette[1], show=False)

fig = make_subplots(rows=1, cols=2, subplot_titles=['train', 'test'])
    
fig.add_trace(fund_total_train_tr, row=1, col=1)
fig.add_trace(fund_total_test_tr, row=1, col=2)

fig.update_layout(
    height=600,
    width=1000,
    title='funding_total_usd boxplots'
)

fig.show()

display(df_train['funding_total_usd'].describe().to_frame().T.style.set_caption('train set'), 
        df_test['funding_total_usd'].describe().to_frame().T.style.set_caption('test set'))

In [ ]:
def drop_rows_funding_total(df):
    rows_to_delete = df[df['funding_total_usd'] > 5e9].index
    print(f'Startups with total fundings greater than 5b $: {rows_to_delete.shape[0]}')
    df = df.drop(index = rows_to_delete)
    return df

df_train = drop_rows_funding_total(df_train)
# df_test  = drop_rows_funding_total(df_test)
print(df_train.shape, df_test.shape)

Графики выше показывают, что имеются выбивающиеся суммы инвестиций. Удалим все стартапы, суммарные инвестиции в которые превышают 5 миллиардов долларов.

In [ ]:
# print('Number of startupds with total fundings lower than 10k$: ', df_train.query('funding_total_usd < 10000').shape[0])
# df_train = df_train.drop(index=df_train.query('funding_total_usd < 10000').index)
# df_train.shape

In [ ]:
def show_col_distr_num(df, col, color=palette[0], title='', show=True):
    fig = go.Figure()
    
    trace = go.Histogram(x=df[col], name='', marker_color=color, showlegend=False)
    fig.add_trace(trace)
    
    fig.update_layout(
        height=600,
        width=800,
        title=title,
    )

    if show:
        fig.show()

    return trace

In [ ]:
fund_total_train_tr = show_col_distr_num(df_train, 'funding_total_usd', show=False)
fund_total_test_tr  = show_col_distr_num(df_test,  'funding_total_usd', color=palette[1], show=False)

fig = make_subplots(rows=1, cols=2, subplot_titles=['train', 'test'])
    
fig.add_trace(fund_total_train_tr, row=1, col=1)
fig.add_trace(fund_total_test_tr, row=1, col=2)

fig.update_layout(
    height=600,
    width=1000,
    title='funding_total_usd distributions'
)

fig.show()

### country_code / state_code / region / city

In [ ]:
df_train['country_code'].unique() 

In [ ]:
df_train['region'].nunique()

In [ ]:
df_train['city'].nunique()

In [ ]:
df_train['state_code'].sort_values().unique()

In [ ]:
# Страны, у которых есть своя собственная кодировка штатов/регионов
df_train[df_train['state_code'].notna()]['country_code'].unique()

In [ ]:
df_train[df_train['state_code'].isna()].groupby('country_code')['state_code'].apply(lambda x: x.isna().sum()).sort_values(ascending=False).to_frame().T

### category_list

В связи с наличием комбинированных категорий необходимо упростить их до одной категории, чтобы в дальнейшем было возможно присвоить стартапу соответсвтующую уникальную группу. Для этого определим список рейтинг популярности одиночных категорий, после чего вместо комбинированной категории оставим лишь одну категорию, которая в рейтинге стоит выше.

In [ ]:
category_list_splitted = [item.split('|') for item in df_train.category_list.fillna('').str.lower()]
category_counts = Counter(category for sublist in category_list_splitted for category in sublist)

In [ ]:
def get_main_category(df, category_counts):
    category_list_splitted = [item.split('|') for item in df['category_list'].fillna('').str.lower()]
    main_categories_list   = [[category for category in sorted(sublist, key=lambda x: -category_counts.get(x, 0))][0] for sublist in category_list_splitted]
    return main_categories_list

In [ ]:
df_train['main_category'] = get_main_category(df_train, category_counts)
df_test['main_category']  = get_main_category(df_test, category_counts)
print(df_train.shape, df_test.shape)
display(df_train.head())

### status

In [ ]:
df_train['status'].value_counts()

---

# EDA

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.heatmap(df_train.isnull(), cbar=False, ax=axes[0])
axes[0].set_title('Missing values in df_train')

sns.heatmap(df_test.isnull(), cbar=False, ax=axes[1])
axes[1].set_title('Missing values in df_test')

plt.show()

Из графика выше можно выделить, что в большинстве случаев пропуски в данных, связанных с географическим происхождением старапов, имеются сразу во всех соответствующих столбцах. Это означает, что восстановить какую-либо информацию по косвенным признакам скорее всего будет крайне затруднительно в связи с отсутствием сразу всех географических данных.

In [ ]:
def plot_size(df, column, labels, explode, palette):
    values = df[column].value_counts()
    display(values.to_frame())
    
    lb = ''
    if labels == '':
        lb = values.index
    else:
        lb = labels
    
    fig, ax = plt.subplots(2, 1, figsize=(10, 10), tight_layout=True)

    ax[0].bar(lb, values, color=palette)
    ax[0].grid(True, color='grey', axis='y', linestyle='-.', linewidth=0.5, alpha=0.6)
    ax[0].set_xlabel('Class', fontsize=16)
    ax[0].set_ylabel('Number of clients', fontsize=14)
    ax[0].set_title(f'Number of clients over "{column}" attribute', fontsize=14)
    ax[0].bar_label(ax[0].containers[0], \
                 label_type='center', fmt='%.2f', fontsize=14, color='white')
    ax[0].tick_params(axis='x', labelsize=12)
    ax[0].tick_params(axis='y', labelsize=12)
    
    pie, _, _ = ax[1].pie(
                            values, 
                            labels=values.index, 
                            autopct='%1.1f%%', 
                            startangle=150, 
                            textprops={'fontsize': 12},
                            colors=sns.color_palette('Set2'),
                            explode=explode,
                            # wedgeprops={'width': 0.4}  # Adjust the width to change the size of the hole
                        )
    plt.setp(pie, edgecolor='black', linewidth=0.3)
    ax[1].set_title('Distribution of Startup Status', fontsize=15, pad=10)
    ax[1].axis('equal')
    ax[1].set_title(f'Percentage of clients over "{column}" attribute ', fontsize=16)

    plt.show()
    
    return

In [ ]:
def plot_tab(df, column, slice=10, filter_value=100, sort_by_ratio=False, horizontal=False):
    fig, ax = plt.subplots(figsize=(6, 5))
    
    tab = pd.crosstab(df[column], df['status'])
    tab['ratio'] = tab['closed'] / (tab['closed'] + tab['operating']) * 100
    if sort_by_ratio:
        tab = tab[tab['operating'] > filter_value]
        tab = tab.sort_values(by=['ratio'], ascending=False)
        
    display(tab[:slice])
    tab = tab.drop(columns=['ratio'])

    if horizontal:
        tab.div(tab.sum(axis=1), axis=0)[slice::-1].plot(kind="barh", stacked=True, color=[palette[1], palette[2]], ax=ax)
        ax.set_xlabel('Proportion', fontsize=12)
        ax.set_ylabel(column, fontsize=12)
        ax.grid(True, color='grey', axis='x', linestyle='-.', linewidth=0.5, alpha=0.6)
        ax.set_xlim(0, 1)
        ax.set_xticks([0.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0])        
        ax.legend(title='status', loc='upper left', labels=['closed', 'operating'], bbox_to_anchor=(1, 1))
        ax.axvline(x=0.5, color='black', linestyle='--', alpha=0.7)
    else:
        tab.div(tab.sum(axis=1), axis=0)[:slice].plot(kind="bar", stacked=True, color=[palette[1], palette[2]], ax=ax)
        ax.set_xlabel(column)
        ax.set_ylabel('Proportion')
        ax.grid(True, color='grey', axis='y', linestyle='-.', linewidth=0.5, alpha=0.6)
        ax.legend(title='status', loc='upper left', labels=['closed', 'operating'], bbox_to_anchor=(1, 1))

    plt.xticks(rotation=0)
    ax.bar_label(ax.containers[0], label_type='center', fmt='%.2f')
    ax.bar_label(ax.containers[1], label_type='center', fmt='%.2f')
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_title(f'Stacked Bar Chart of {column} vs. status')
    
    plt.show()
    
    return

In [ ]:
plot_size(df_train, 'status', labels='', explode=(0, 0.05), palette=sns.color_palette('Set2'))

Имеется ярко выраженный дисбаланс классов. Закрывшихся стартапов в выборке в 10 раз меньше, чем стартапов, продолжающих функционировать.

In [ ]:
_ = show_col_distr_cat(df_train, 'country_code', slice=15, stacking_col='status',
                       title='Число стартапов по странам', x_axis_t='страны', y_axis_t='кол-во',
                       color=palette[0])

In [ ]:
print(f'Доля стартапов из США: {df_train[df_train['country_code'] == 'USA'].shape[0] / df_train.shape[0] * 100:.2f}%')
print(f'Доля стартапов из Великобритании: {df_train[df_train['country_code'] == 'GBR'].shape[0] / df_train.shape[0] * 100:.2f}%')
print(f'Доля стартапов из Великобритании: {df_train[df_train['country_code'] == 'CAN'].shape[0] / df_train.shape[0] * 100:.2f}%')

In [ ]:
plot_tab(df_train, 'country_code', slice=15, filter_value=50, sort_by_ratio=True, horizontal=True)

In [ ]:
def plot_pivot_table(df, index, closed=False, filter_value=100, title='', x_axis_t='', y_axis_t='', color=palette[0], orientation='v'):
    pivot_table = df.pivot_table(index=index, columns='status', aggfunc='size', fill_value=0)
    pivot_table.columns = ['closed', 'opened']
    pivot_table = pivot_table[pivot_table['opened'] > filter_value] # уберем совсем редкие категории стартапов
    ratio = ''
    if closed:
        ratio = pd.DataFrame(
                              data=[c / (o + c) * 100 for o, c in zip(pivot_table['opened'], pivot_table['closed'])], 
                              index=pivot_table.index,
                              columns=['ratio']
                            ).sort_values(by=['ratio'], ascending=False)
    else:
        ratio = pd.DataFrame(
                              data=[o / (o + c) * 100 for o, c in zip(pivot_table['opened'], pivot_table['closed'])], 
                              index=pivot_table.index,
                              columns=['ratio']
                            ).sort_values(by=['ratio'], ascending=False)
    display(ratio.head(10).T)
    
    fig = go.Figure()
    if orientation == 'v':
        trace = go.Bar(x=ratio.index, y=ratio.ratio, name='', marker_color=color, showlegend=False, opacity=0.8)
    elif orientation == 'h':
        trace = go.Bar(y=ratio.index[::-1], x=ratio.ratio[::-1], name='', marker_color=color, orientation=orientation, showlegend=False, opacity=0.8)
    fig.add_trace(trace)
    
    fig.update_layout(
        height=600,
        width=1000,
        title=title,
        xaxis_title = x_axis_t if orientation=='v' else y_axis_t,
        yaxis_title = y_axis_t if orientation=='v' else x_axis_t,
    )
    
    fig.show()

In [ ]:
plot_pivot_table(df_train, 
                 'country_code', 
                 closed=True, 
                 filter_value=50, 
                 title='Доля провальных стартапов по странам', 
                 x_axis_t='страны', 
                 y_axis_t='доля',
                 orientation='h',
                 color=palette[1])

Страной, в которой было зафиксировано самое большое число стартапов, являются США. На стартапы из США приходится более половины всех записей в выборке - почти 57%. Великобритания находится на втором месте по числу стартапов, но ее доля в 10 раз меньше, чем у США - 5.57%. Далее по нисходящей. При этом, что примечательно, доля провальных стартапов в США меньше, чем во многих других странах. США находится приблизительно посередине по показателю доли закрывшихся стартапов.

Странами с самым большим числом закрывшихся стартапов являются Россия(доля провалов равна 35.5%), Бразилия(11.8%), Новая Зеландия(11.7%), Аргентина и Швеция(11.68% и 11.26% соответственно). В данном случае были учтены страны, в которых было открыто более 50 стартапов.

Меньше всего стартапов было свернуто в Португалии, Чили, ОАЭ, Индии и Турции.

In [ ]:
_ = show_col_distr_cat(df_train, 'city', slice=30, stacking_col='status',
                       title='Число стартапов по городам', x_axis_t='города', y_axis_t='кол-во',
                       color=palette[0])

In [ ]:
df_train_cat = df_train[df_train['main_category'] != '']

In [ ]:
_ = show_col_distr_cat(df_train_cat, 'main_category', slice=15, stacking_col='status',
                       title='Число стартапов по категориям', x_axis_t='категории', y_axis_t='кол-во',
                       color=palette[0])

In [ ]:
plot_tab(df_train_cat, 'main_category', slice=10, sort_by_ratio=True, horizontal=True)

In [ ]:
plot_pivot_table(df_train_cat, 
                 'main_category', 
                 filter_value=100, 
                 closed=False, 
                 title='Доля функционирующих стартапов по категориям', 
                 x_axis_t='категория', 
                 y_axis_t='доля, %', 
                 color=palette[0])

In [ ]:
plot_pivot_table(df_train_cat, 
                 'main_category', 
                 filter_value=100, 
                 closed=True, 
                 title='Доля провальных стартапов по категориям', 
                 x_axis_t='категория', 
                 y_axis_t='доля, %', 
                 color=palette[1])

del df_train_cat

Графики выше демонстрируют отношение направленности стартапа к числу провальных стартапов в данной области, а также в целом распределение записей по категориям.

Если говорить о распределении по категориям, то здесь стоит отметить категорию "software" как самую распространенную в выборке. Число стартапов, соответствующих данной категории, почти в 2 раза больше размеров следующей по популярности категории. В пятерку самых распространенных категорий входят: software, biotechnology, mobile, e-commerce, curated web. Следом идут enterprise software, healthcare и games. В целом среди самых популярных стартапов превалируют категории, связанные с цифровой или технической сферой.

Для некоторых из самых популярных категорий стартапов наблюдается также и более высокая доля провальных бизнесов. Например, curated web является 5 по распространенности категорией, будучи при этом 2 среди по доле провальных случаев. Games - 8 по популярности и 11 по проценту провалов категория. Software - 1 по популярности и 22 по проценту провалов. И т.д. В целом это ожидаемо: чем больше молодых стартапов пытается развиться, тем в общем больше шансов, что кого-нибудь постигнет неудача.

In [ ]:
train_tr = show_col_distr_cat(df_train, 'funding_rounds', color=palette[0], title='Funding rounds distribution for train set', show=False)
test_tr  = show_col_distr_cat(df_test,  'funding_rounds', color=palette[1], title='Funding rounds distribution for test set', show=False)

fig = make_subplots(rows=1, cols=2, subplot_titles=['train', 'test'])
    
fig.add_trace(train_tr, row=1, col=1)
fig.add_trace(test_tr, row=1, col=2)

fig.update_layout(
    height=600,
    width=1000,
    title='funding_total_usd distributions',
    xaxis=dict(dtick=1)
)

fig.show()

In [ ]:
train_fig.show()

In [ ]:
df_status = pd.DataFrame(df_train[['status', 'funding_rounds']])
pivot_table = df_status.pivot_table(index='funding_rounds', columns='status', aggfunc='size', fill_value=0)
pivot_table.columns = ['closed', 'opened']
close_ratio = pd.Series(
                        data=[c / (o + c) * 100 for o, c in zip(pivot_table['opened'], pivot_table['closed'])], 
                        index=pivot_table.index
                       )

display(pivot_table.T)
display(close_ratio.to_frame().T.style.background_gradient(cmap='YlOrRd', axis=1))

sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
fig = plt.figure(figsize=(8, 6))
sns.lineplot(x=close_ratio.index, y=close_ratio.values)
plt.xticks(range(0, len(close_ratio) + 1, 1))
plt.ylabel('%', fontsize=12)
plt.xlabel('Funding Rounds', fontsize=12)
plt.title('Отношение числа закрытых стартапов к общему числу стартапов по числу раундов финансирования', fontsize=14)
plt.show()

- В выборке представлены только стартапы, получившие финансирование хотя бы один раз.
- Имеется некоторая вполне ожидаемая тенденция к снижению числа провальных стартапов с каждым следующим раундом финансирования. Такое поведение справедливо вплоть до 8 раунда инвестиций. График выше показывает некоторое несоответствие общему тренду отношения числа закрытых стартапов к общему их числу в промежутке между 7 и 12 раундом. Но стоит учитывать, что это несущественная проблема, учитывая маштаб общего числа стартапов, прошедших данные итерации финансирования. С 12 раунда и далее число закрывшихся стартапов равно 0.

In [ ]:
df_train['funding_total_usd'].describe().to_frame().T

In [ ]:
df_log_fund_train = pd.DataFrame(data=df_train['funding_total_usd'].apply(np.log))
                                                                         
df_log_fund_test  = pd.DataFrame(data=df_test['funding_total_usd'].apply(np.log))

df_log_fund_train.columns = ['log_funding_total']
df_log_fund_test.columns = ['log_funding_total']

display(df_log_fund_train.describe().T, df_log_fund_test.describe().T)

In [ ]:
def plot_scatter_class(df, col, title=''):
    opened = df[df['status'] == 'operating'][col]
    closed = df[df['status'] == 'closed'][col]

    sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
    plt.figure(figsize=(10, 6))
    
    plt.scatter(x=opened, y=[0] * opened.shape[0])
    plt.scatter(x=closed, y=[1] * closed.shape[0])

    plt.yticks([0, 1], ['Operating', 'Closed'])
    plt.title(title, fontsize=14)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('status', fontsize=12)
    
    plt.show()

In [ ]:
df_log_fund_train['status'] = df_train['status']

plot_scatter_class(df_log_fund_train, 'log_funding_total', title='Разброс log_funding_total по статусу стартапа')

In [ ]:
def show_distr_sns(df, col, color=palette[0], title=''):
    plt.figure(figsize=(10, 6))
    
    sns.histplot(df[col], kde=True, color=color)
    
    plt.title(title)
    plt.xlabel(col)
    plt.ylabel('Frequency')
    
    plt.show()

In [ ]:
# fund_total_train_tr = show_col_distr_num(df_log_fund_train, 'log_funding_total', show=False)
# fund_total_test_tr  = show_col_distr_num(df_log_fund_test,  'log_funding_total', color=palette[1], show=False)

show_distr_sns(df_log_fund_train, 'log_funding_total', title='Распределение funding_total_usd после log-преобразования (трейн)')
show_distr_sns(df_log_fund_test,  'log_funding_total', color=palette[1], title='Распределение funding_total_usd после log-преобразования (тест)')

In [ ]:
fund_total_train_tr = plot_boxplot(df_log_fund_train, 'log_funding_total', show=False)
fund_total_test_tr  = plot_boxplot(df_log_fund_test,  'log_funding_total', color=palette[1], show=False)

fig = make_subplots(rows=1, cols=2, subplot_titles=['train', 'test'])
    
fig.add_trace(fund_total_train_tr, row=1, col=1)
fig.add_trace(fund_total_test_tr, row=1, col=2)

fig.update_layout(
    height=600,
    width=1000,
    title='log funding_total_usd boxplots'
)

fig.show()

In [ ]:
def find_outliers(data):
    series = pd.Series(data)
    
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    
    IQR = Q3 - Q1
    
    LF = Q1 - 1.5 * IQR
    UF = Q3 + 1.5 * IQR
    
    outliers = series[(series < LF) | (series > UF)]
    return outliers

In [ ]:
print('Выбросов до преобразования: ', len(find_outliers(df_train['funding_total_usd'])))
print('Выбросов после преобразования: ', len(find_outliers(df_log_fund_train['log_funding_total'])))

In [ ]:
def perform_normality_tests(data):
    statistic, pvalue = st.jarque_bera(data)
    if pvalue > 0.05:
        result = 'Normal'
    else:
        result = 'NOT Normal'
    print(f'Jarque-Bera: {result:>20s}')
    print(statistic)
    print(f'p-value: {pvalue}')

    ksstat, pvalue = sm.stats.diagnostic.lilliefors(data)
    if pvalue > 0.05:
        result = 'Normal'
    else:
        result = 'NOT Normal'
    print(f'Lilliefors: {result:>30}')
    print(ksstat)
    print(f'p-value: {pvalue}')

    statistic, pvalue = st.normaltest(data)
    if pvalue > 0.05:
        result = 'Normal'
    else:
        result = 'NOT Normal'
    print(f'Normaltest: {result:>20s}')
    print(statistic)
    print(f'p-value: {pvalue}')

    return

In [ ]:
perform_normality_tests(df_log_fund_train['log_funding_total'].dropna())

In [ ]:
perform_normality_tests(df_log_fund_test['log_funding_total'].dropna())

In [ ]:
del df_log_fund_train
del df_log_fund_test

Т.к. ранее в процессе предобработки был замечен сильный скос графика распределения признака вправо, использовано log-преобразование, чтобы получить более плотное распределение с более "приятными" статистическими свойствами, в котором нет значительных выбросов, как это было в данных до преобразования. Хотя полученное распределение издалека напоминает нормальное распределение, тесты на нормальность показывают, что это не так. Но даже так полученное распределение выглядит намного лучше изначального. Кроме того, после преобразования количество выбросов, не попадающих в полтора межквартильных размаха, уменьшилось с 5874 до 158.

In [ ]:
foundation_stats_train = df_train.groupby(df_train['founded_at'].dt.year)['funding_rounds'].count()
foundation_stats_test  = df_test.groupby(df_test['founded_at'].dt.year)['funding_rounds'].count()

display(foundation_stats_train.to_frame().T.style.set_caption('train set'))
display(foundation_stats_test.to_frame().T.style.set_caption('test set'))

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

sns.lineplot(x=foundation_stats_train.index, y=foundation_stats_train.values, ax=ax[0])
sns.lineplot(x=foundation_stats_test.index,  y=foundation_stats_test.values,  ax=ax[1])

ax[0].set_title('Число открывшихся стартапов по годам (train)')
ax[1].set_title('Число открывшихся стартапов по годам (test)')

plt.show()

In [ ]:
df_train_year = df_train.groupby([df_train['founded_at'].dt.year, 'status']).size().unstack(level='status').fillna(0)
df_train_year = df_train_year[df_train_year.columns[::-1]]
display(df_train_year.T)

sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})

df_train_year.plot(kind='bar', stacked=True, figsize=(10, 6), width=0.8, color=palette)

plt.xlabel('Год')
plt.ylabel('Число стартапов')
plt.title('Состояние стартапов по годам')

plt.show()

# fig = go.Figure()
# for i, col in enumerate(df_train_year.columns):
#     local_distr = df_train_year[col]
#     local_trace = go.Bar(x=local_distr.index, y=local_distr.values, name=col, marker_color=palette[i], showlegend=True)
#     fig.add_trace(local_trace)

# fig.update_layout(
#     barmode='stack',
#     height=600,
#     width=800,
#     title='Состояние стартапов по годам',
# )

# fig.show()

In [ ]:
growth = [(new_year - prev_year) / prev_year * 100 for prev_year, new_year in zip(foundation_stats_train[0::], foundation_stats_train[1::])]
growth = pd.Series(data=growth[:-1], index=foundation_stats_train.index[1:-1])

plt.figure(figsize=(12,6))
sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})
plt.title('Процентный рост числа стартапов в сравнении с прошлым годом', size=16)
plt.ylabel('Рост, %',size=12)
plt.xlabel('Год',size=12)
plt.axhline(0, linestyle='-.', color='red')
plt.plot(growth.index, growth.values, label='Рост',color='green',linewidth=3)
plt.xticks(growth.index[::5])
plt.legend()
plt.show()

In [ ]:
close_ratio = pd.Series(
                        data=[c / (o + c) * 100 for o, c in zip(df_train_year['operating'], df_train_year['closed'])], 
                        index=df_train_year.index
                       )[:-1]

display(close_ratio.to_frame().T)

plt.figure(figsize=(12,6))
sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})
plt.title('Доля провальных стартапов по годам основания', size=16)
plt.ylabel('% провалов',size=12)
plt.xlabel('Год',size=12)
plt.plot(close_ratio.index, close_ratio.values, label='Процент провалов',color='red',linewidth=3)
plt.axhline(close_ratio.values.mean(), label='Средний уровень провалов за всё время', linestyle='-.')
plt.legend()
plt.xticks(close_ratio.index[::5])
plt.yticks(range(0, int(max(close_ratio.values)) + 1, 2))
plt.show()

Графики выше предназначены больше для ознакомления с датасетом, нежели для нахождения закономерностей, полезных для предсказательной модели.

Как можно заметить, с каждым годом число новых стартапов росло, особенно резко рост начался приблизительно 1995-96 году, когда число стратапов начало увеличиваться экспоненциально. На графике также видно две условные ступени, когда рост тормозился. Такие проблемы возникли в 2000 и 2008 году, что, разумеется, имеет исторический контекст (dot-com bubble и мировой экономический кризис, начавшийся в 2008 году).

График отношения числа провальных стартапов к общему числу стартапов по году создания стартапа имеет скачкообразное поведение вплоть до 1985-88 года. Связано это с небольшим числом стартапов, открывшихся в этот период. Более устойчивое и читаемое поведение графика начинается далее в связи с ростом числа стартапов. До 1997 года провальных стартапов в среднем было менее 10%. С 97 по 2006-2007 данное соотношение.

In [ ]:
df_founded_before_funding = df_train[df_train['founded_at'].dt.year <= df_train['first_funding_at'].dt.year].copy(deep=True)
df_founded_after_funding  = df_train[df_train['founded_at'].dt.year >  df_train['first_funding_at'].dt.year].copy(deep=True)

df_founded_before_funding['year_of_foundation'] = df_founded_before_funding['founded_at'].dt.year
df_founded_after_funding['year_of_foundation']  = df_founded_after_funding['founded_at'].dt.year
df_founded_after_funding  = df_founded_after_funding[df_founded_after_funding['year_of_foundation'] != 2016]
df_founded_before_funding = df_founded_before_funding[df_founded_before_funding['year_of_foundation'] 
                                                      >= df_founded_after_funding['year_of_foundation'].min()]

pivot_table_before = df_founded_before_funding.pivot_table(index='year_of_foundation', columns='status', aggfunc='size', fill_value=0)
pivot_table_after  = df_founded_after_funding.pivot_table(index='year_of_foundation',  columns='status', aggfunc='size', fill_value=0)

pivot_table_before.columns = ['closed', 'opened']
pivot_table_after.columns  = ['closed', 'opened']

display(pivot_table_before.T.style.set_caption('founded before funding'), 
        pivot_table_after.T.style.set_caption('founded after funding'))

close_ratio_before = pd.Series(
                                data=[c / (o + c) * 100 for o, c in zip(pivot_table_before['opened'], pivot_table_before['closed'])], 
                                index=pivot_table_before.index
                              )
close_ratio_after  = pd.Series(
                                data=[c / (o + c) * 100 for o, c in zip(pivot_table_after['opened'], pivot_table_after['closed'])], 
                                index=pivot_table_after.index
                              )


display(close_ratio_before.to_frame().T.style.set_caption('founded before funding, ratio'), 
        close_ratio_after.to_frame().T.style.set_caption('founded after funding, ratio'))
df_combined = pd.concat([close_ratio_before, close_ratio_after], axis=1)
df_combined.columns = ['before funding', 'after funding']
df_combined.loc[1997, 'after funding'] = 0.0
df_combined['year_of_foundation'] = df_combined.index
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.melt(id_vars='year_of_foundation', var_name='Category', value_name='Ratio')

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")
sns.barplot(x='year_of_foundation', y='Ratio', hue='Category', data=df_combined, palette='muted')

# Customize the plot
plt.xlabel('Year of Foundation', fontsize=12)
plt.ylabel('Ratio (%)', fontsize=12)
plt.title('Сравнение % закрывшихся стартапов среди основанных до или после первого раунда финансирования', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Founded')

# Show plot
plt.tight_layout()
plt.show()

График выше сравнивает отношение числа закрывшихся - провальных - стартапов к общему числу стартапов, созданных в каждом году. При этом сравнивается отношение двух подвыборок. Первая является подвыборкой стартапов, получивших свое первое финансирование еще до своего основания (оранжевые столбцы). Вторая подвыборка наоборот состоит из тех стартапов, у которых первый раунд финансирования прошел уже после основания фирмы (синие столбцы). Здесь стоит отметить несколько моментов. Во-первых, выборка содержит относительно небольшое число записей, относящихся к первой подвыборке - всего около 1200 строк при общем размере тестового датасета в более чем 50000 строк. Во-вторых, стартапы первой подвыборки равномерно распределены по годам основания лишь с 2005 года. До этого момента примеры таких бизнесов встречаются лишь в 1999 году.

Если сравнивать проценты провальных стартапов лишь в те года, для которых имеется информация об обеих подвыборках, то можно заметить некоторое непостоянство. Ровно в половине случаев стартапы с "превентивным" финансированием проваливались заметно (иногда более чем в 2 раза) чаще второй категории стартапов. Однако в другой половине случаев стартапы, получавшие финансирование уже после своего создания, проваливались чаще первой категории. Хотя в данном случае разница в процентном отношении не такая впечатляющая, как в первой случае: разница варьируется от менее чем 0.1% до приблизительно 5%, лишь в 2015 году разница составила порядка 12%, в то время как в случае большего числа провалов среди стартапов первой группы разница находится в интервале от 1.76% до 18%.

Говорить о существовании какой-либо закономерности в данном случае тяжело, т.к. не удалось установить однозначной тенденции к более частому провалу среди рассмотренных категорий стартапов. Единственное, что можно с натяжкой определить как существующую разницу - более высокую долю провалов стартапов, профинансированных до своего основания, в случаях, когда выдавался более провальный год именно для них. Однако, что более вероянтно, такая разница связана с размерами подвыборок. Всё-таки выборка таких стартапов значительно меньше второй категории, что заставляет сомневаться в статистической устойчивости полученных наблюдений.

Если учитывать всё скананное ранее, проводить подобную дихотомию скорее является излишниш и вряд ли приведет к дальнейшему улучшению модели. Однако можно рассмотреть возможность проведения экспериментов, в рамках которых будут сравниваться модели, обученные на выборках с новым признаком, классифицирующим стартапы описанным выше способом, и без такого признака. 

In [ ]:
first_last_funding_gap = df_train['last_funding_at'] - df_train['first_funding_at']
first_last_funding_gap = first_last_funding_gap.dt.days
first_last_funding_gap = first_last_funding_gap.dropna().astype(int)
first_last_funding_gap = pd.DataFrame(data=first_last_funding_gap.values, index=first_last_funding_gap.index, columns=['range']).sort_values(by='range')
first_last_funding_gap = first_last_funding_gap[first_last_funding_gap['range'] > 0]
first_last_funding_gap['status'] = df_train['status']
first_last_funding_gap.shape

In [ ]:
plot_scatter_class(first_last_funding_gap, 'range', title='Разброс времени между первым и последним раундом финансирования по статусу стартапа')

In [ ]:
tmp = df_train[df_train['last_funding_at'] - df_train['first_funding_at'] > pd.Timedelta(3000, 'd')][['funding_rounds', 'status']]
tmp.groupby('status').count()

In [ ]:
tmp['funding_rounds'].value_counts().to_frame().T

Анализ времени между первым и последним раундом финансирования показывает, что возможно установить такой порог для временной дельты, после которого останутся только функционирующие стартапы. Однако доля выделенных записей совсем незначительна, так что особой пользы от подобного заключения получить вряд ли получится.

---

## Feature engineering

In [ ]:
df_train = df_train.reset_index(drop=True)

### category_list

In [ ]:
# unique_cats = sorted(df_train.category_list.str.lower().str.replace(' ', '').str.split('|').explode().str.strip().dropna().unique())
unique_cats = sorted(df_train.category_list.str.lower().str.split('|').explode().str.strip().dropna().unique())

In [ ]:
len(unique_cats)

In [ ]:
print(unique_cats[:10], unique_cats[-10:])

Создадим, пока в отдельной переменной, список, содержащий разбитый список категорий для каждой строки в датасете. После этого, если стартап имеет более одной категории, отсортируем локальный список по популярности категорий.

In [ ]:
# # Перенес в предобработку
# category_list_splitted = [item.split('|') for item in df_train.category_list.fillna('').str.lower()]

category_list_splitted[:5], len(category_list_splitted)

In [ ]:
# # Перенес в предобработку
# category_counts = Counter(category for sublist in category_list_splitted for category in sublist)

category_list_splitted = [[category for category in sorted(sublist, key=lambda x: -category_counts.get(x, 0))] for sublist in category_list_splitted]
category_list_splitted[:5]

In [ ]:
# второе условие - аналог проверки на nan, т.к. выше я использовал fillna('')
rows_with_single_cat = pd.Index([i for i, lst in enumerate(category_list_splitted) if (len(lst) == 1) and ('' not in lst)])
print('Кол-во стартапов, характеризующихся единственной категорией: ', rows_with_single_cat.shape[0])

Добавим новый бинарный признак, характеризующий стартап как многонаправленный или с единственным профилем в зависимости от числа категорий в поле **category_list**. Для тех записей, в которых имеются пропуски в поле category_list, будем назначать категорию, соответствующую единственному профилю. Назовем новое поле **multidisciplinary**

In [ ]:
df_train['multidisciplinary'] = [False if i in rows_with_single_cat else True for i in df_train.index]
df_train['multidisciplinary'].value_counts()

Код для получения данных непосредственно со страницы сайта поддержки.

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup

# # URL of the web page
# url = 'https://support.crunchbase.com/hc/en-us/articles/360043146954-What-Industries-are-included-in-Crunchbase'

# # Initialize a headless browser
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Run in headless mode, no browser window
# options.add_argument('--disable-gpu')  # Disable GPU acceleration
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# # Fetch the HTML content of the web page
# driver.get(url)
# html_content = driver.page_source

# # Close the browser
# driver.quit()

# # Parse the HTML
# soup = BeautifulSoup(html_content, 'html.parser')

# # Find the table containing industry groups and industries
# table = soup.find('table')

# # Initialize an empty dictionary to store industry groups and their corresponding industries
# industry_data = {}

# # Loop through rows in the table
# for row in table.find_all('tr')[1:]:  # Skip the first row (header row)
#     # Extract data from cells in the row
#     cells = row.find_all('td')
#     industry_group = cells[0].text.strip()
#     industries = [industry.strip() for industry in cells[2].text.split(',')]
    
#     # Add industry group and industries to the dictionary
#     industry_data[industry_group] = industries

# # Print the dictionary
# for industry_group, industries in industry_data.items():
#     print(f"{industry_group}: {', '.join(industries)}")

# with open('startup_categories.json', 'w') as json_file:
#     json.dump(industry_data, json_file)

Код для получения уже собранного со страницы поддержки файла с моего гитхаба, чтобы код работал на любой системе

In [ ]:
url = 'https://raw.githubusercontent.com/Lighter01/startups_categories/main/startup_categories.json'

local_file_path = './datasets/startup_categories.json'

if os.path.exists(local_file_path):
    print("File already exists.")
else:
    response = requests.get(url)

    if response.status_code == 200:
        with open(local_file_path, 'wb') as json_file:
            json_file.write(response.content)
        print("JSON file downloaded successfully.")
    else:
        print("Failed to download JSON file.")

В связи с тем, что имеются пересечения по категориям стартапов между новыми обобщенными группами, нужно определить однозначное соответствие между категорией и группой. Т.к. вручную проводить анализ каждой категории на соответствие группе не является рациональным решением, упростим задачу. Определим категорию к той группе, в которой она впервые встретилась, а все последующие включения удалим.

In [ ]:
data = dict()

with open('./datasets/startup_categories.json', 'r') as json_file:
    data = json.load(json_file)

print(sum(len(industries) for industries in data.values()))

In [ ]:
cat_dict = {v.lower().strip(): k.lower().strip() for k, v_list in reversed(data.items()) for v in v_list}
len(cat_dict)

In [ ]:
cat_dict['hardware + software'] = 'hardware/software'
len(cat_dict)

In [ ]:
duplicated_categories = list()
checked_categories = set()

for category_group, category_list in data.items():
    for category in category_list:
        if category in checked_categories:
            duplicated_categories.append((category, category_group))
        else:
            checked_categories.add(category)

duplicated_categories = sorted(duplicated_categories)
print('Кол-во дубликатов категорий в различных группах: ', len(duplicated_categories))

Теперь добавим новый признак - category_group - для обобщения всего разнообразия типов стартапов. Для увеличения шанса найти соответствующую группу, в случае если по главной категории не удастся определить группу, далее будут перебираться другие категории стартапа, если такие имеются, и по первому определенному соответствию будет назначена группа. Категории из списка всех категорий стартапа будут перебираться в порядке уменьшения распостраненности категории. Для всех стартапов, для которых не нашлось соответствующей группы, будем назначать группу 'no_group' по умолчанию.

In [ ]:
def map_category_to_group(row):
    if row[0] == '':
        return ''
    
    category = row[0]
    if category in cat_dict:
        return cat_dict[category]
    
    category_list = row[1]
    for category in category_list:
        if category in cat_dict:
            return cat_dict[category]
    
    return 'no_group'

vfunc = np.vectorize(map_category_to_group)

In [ ]:
dtype = [('main_category', object), ('category_list', object)]

cat_cat_list = np.empty(df_train.shape[0], dtype=dtype)

cat_cat_list['main_category'] = df_train['main_category']
cat_cat_list['category_list'] = category_list_splitted

In [ ]:
category_group = vfunc(cat_cat_list)
print('Кол-во записей, для которых не удалось установить группу: ', np.sum(category_group == 'no_group'))

In [ ]:
no_group_id = np.where(category_group == 'no_group')[0]
no_group_id.shape

In [ ]:
missing_cats = np.unique([cat for sublist in cat_cat_list[no_group_id]['category_list'] for cat in sublist])
missing_cats.shape

In [ ]:
df_train.loc[no_group_id, 'status'].value_counts().to_frame()

Всего для 6343 записей в тренировочной выборке не удалось определить группу для категории. Анализ этих 6 тысяч записей показал, что из 803 уникальных категорий не нашлось соответствия для 320. Это достаточно много. Однако другой информации о соответствии категории группе не представится, поэтому попробуем решить проблему так. Будем сравнивать оставшиеся неопределенные категории с категориями, для которых имеется соответсвие с группой, сравнивая последовательность символов. Т.е. будем сравнивать строки на близость. Для каждой неопределенной категории найдем максимально схожую категорию из тех, для которой известна группа, и будем присваивать соответствующему стартапу данной категории группу данной наиболее близкой категории.

In [ ]:
missing_cats_list = [sublist for i, sublist in enumerate(category_list_splitted) if i in (no_group_id)]
len(missing_cats_list)

In [ ]:
missing_category_counts = Counter(category for sublist in missing_cats_list for category in sublist)
missing_category_counts = sorted(missing_category_counts.items(), key=lambda x: x[1], reverse=True)

for key, value in missing_category_counts:
    if value > 100:
        print(key, value)

In [ ]:
missing_values_map = dict()
for missing_cat in missing_cats:
    new_key = process.extractOne(missing_cat, cat_dict.keys())
    missing_values_map[missing_cat] = cat_dict[new_key[0]]

missing_values_map

In [ ]:
# Ручные правки
missing_values_map['automated kiosk'] = cat_dict['sales automation']
missing_values_map['all students'] = 'education'
missing_values_map['bridging online and offline'] = 'commerce and shopping'
missing_values_map['building products'] = 'real estate'
missing_values_map['cars'] = 'transportation'
missing_values_map['college campuses'] = cat_dict['college recruiting']
missing_values_map['defense'] = 'privacy and security'
missing_values_map['discounts'] = 'commerce and shopping'
missing_values_map['displays'] = 'consumer electronics'
missing_values_map['doctors'] = 'health care'
missing_values_map['early-stage technology'] = 'financial services'
missing_values_map['english-speaking'] = 'other'
missing_values_map['embedded hardware and software'] = 'hardware/software'
missing_values_map['entertainment industry'] = 'media and entertainment'
missing_values_map['environmental innovation'] = cat_dict['innovation management']
missing_values_map['games'] = 'gaming'
missing_values_map['gold'] = 'natural resources'
missing_values_map['health services industry'] = 'health care'
missing_values_map['heavy industry'] = 'manufacturing'
missing_values_map['hi tech'] = 'information technology'
missing_values_map['high school students'] = 'eduaction'
missing_values_map['home owners'] = 'real estate'
missing_values_map['human resource automation'] = 'professional services'
missing_values_map['iphone'] = 'mobile'
missing_values_map['investment management'] = 'financial services'
missing_values_map['independent pharmacies'] = 'health care'
missing_values_map['reviews and recommendations'] = 'sales and marketing'
missing_values_map['self development'] = 'education'
missing_values_map['senior health'] = 'health care'
missing_values_map['services'] = 'other'
missing_values_map['service providers'] = 'professional services'
missing_values_map['smart grid'] = 'energy'
missing_values_map['television'] = 'media and entertainment'
missing_values_map['renewable tech'] = 'other'
missing_values_map['specialty foods'] = 'food and beverage'
missing_values_map['user experience design'] = cat_dict['ux design']
missing_values_map['tracking'] = 'privacy and security'
missing_values_map['ventures for good'] = 'food and beverage'

In [ ]:
category_group[category_group == 'no_group'] = [missing_values_map[key] for key in cat_cat_list[no_group_id]['main_category']]

In [ ]:
np.sum(category_group == 'no_group')

In [ ]:
df_train['category_group'] = category_group
df_train.shape

In [ ]:
df_train['category_group'].nunique()

In [ ]:
df_train['category_group'].value_counts(ascending=True)[:10]

In [ ]:
small_groups = df_train['category_group'].value_counts()[df_train['category_group'].value_counts() < 20].keys()
df_train.loc[df_train['category_group'].isin(small_groups), 'category_group'] = 'other'

---

Теперь проделаем те же преобразования для тестовой выборки с некоторыми изменениями.

In [ ]:
category_list_splitted_test = [item.split('|') for item in df_test.category_list.fillna('').str.lower()]

In [ ]:
category_list_splitted_test = \
    [[category for category in sorted(sublist, key=lambda x: -category_counts.get(x, 0))] for sublist in category_list_splitted_test]

In [ ]:
# второе условие - аналог проверки на nan, т.к. выше я использовал fillna('')
rows_with_single_cat = pd.Index([i for i, lst in enumerate(category_list_splitted_test) if (len(lst) == 1) and ('' not in lst)])
print('Кол-во стартапов, характеризующихся единственной категорией: ', rows_with_single_cat.shape[0])

In [ ]:
df_test['multidisciplinary'] = [False if i in rows_with_single_cat else True for i in df_test.index]
df_test['multidisciplinary'].value_counts()

In [ ]:
cat_cat_list = np.empty(df_test.shape[0], dtype=dtype)

cat_cat_list['main_category'] = df_test['main_category']
cat_cat_list['category_list'] = category_list_splitted_test

In [ ]:
category_group = vfunc(cat_cat_list)
print('Кол-во записей, для которых не удалось установить группу: ', np.sum(category_group == 'no_group'))

In [ ]:
no_group_id = np.where(category_group == 'no_group')[0]
no_group_id.shape

In [ ]:
missing_cats = np.unique([cat for sublist in cat_cat_list[no_group_id]['category_list'] for cat in sublist])
missing_cats.shape

In [ ]:
df_train.loc[no_group_id, 'status'].value_counts().to_frame()

In [ ]:
missing_cats_list = [sublist for i, sublist in enumerate(category_list_splitted_test) if i in no_group_id]
len(missing_cats_list)

In [ ]:
missing_category_counts = Counter(category for sublist in missing_cats_list for category in sublist)
missing_category_counts = sorted(missing_category_counts.items(), key=lambda x: x[1], reverse=True)

for key, value in missing_category_counts:
    if value > 30:
        print(key, value)

In [ ]:
missing_values_map = dict()
for missing_cat in missing_cats:
    new_key = process.extractOne(missing_cat, cat_dict.keys())
    missing_values_map[missing_cat] = cat_dict[new_key[0]]

missing_values_map

In [ ]:
category_group[category_group == 'no_group'] = [missing_values_map[key] for key in cat_cat_list[no_group_id]['main_category']]

In [ ]:
np.sum(category_group == 'no_group')

In [ ]:
category_group[no_group_id[:10]]

In [ ]:
df_test['category_group'] = category_group
df_test.shape

In [ ]:
df_test['category_group'].value_counts(ascending=True)[:10]

In [ ]:
small_groups = df_test['category_group'].value_counts()[df_test['category_group'].value_counts() < 20].keys()
df_test.loc[df_test['category_group'].isin(small_groups), 'category_group'] = 'other'

Удалим наконец колонку "category_list".

In [ ]:
df_train = df_train.drop(columns=['category_list'])
df_test  = df_test.drop(columns=['category_list'])

df_train.shape, df_test.shape

---

In [ ]:
# missing_values_map_tmp = dict()

# for missing_cat in missing_cats:
#     new_key_1 = process.extractOne(missing_cat, cat_dict.keys())
#     new_key_2 = process.extractOne(missing_cat, cat_dict.keys(), scorer=fuzz.token_set_ratio)
#     missing_values_map_tmp[missing_cat] = (new_key_1[0], new_key_2[0])

# df_missing_values_info_tmp = pd.DataFrame.from_dict(missing_values_map_tmp, orient='index', columns=['without_scorer', 'with_set_scorer'])

---

### funding_total_usd

Проведем лог-преобразование, как делали это ранее в исследовательском анализе, но в этот раз уже сохраним преобразование в датасетах.

In [ ]:
df_train['log_funding_total_usd'] = np.log(df_train['funding_total_usd'])
df_test['log_funding_total_usd']  = np.log(df_test['funding_total_usd'])

df_train.shape, df_test.shape

Добавим еще один признак - среднюю сумму инвестиций. Данный признак будет сильно коррелировать с признаком funding_total_usd, т.к. более половины всех записей - о стартапах, у которых было проведено не более одного раунда финансирования. На этапе корреляционного анализа сравним корреляцию нового признака и funding_total_usd с целевым параметром и выберем тот, который будет более информативным.

In [ ]:
df_train['mean_funding_sum'] = df_train['funding_total_usd'] / df_train['funding_rounds']
df_test['mean_funding_sum']  = df_test['funding_total_usd'] / df_test['funding_rounds']

df_train.shape, df_test.shape

In [ ]:
df_train.head()

---

### founded_at

Выделим отдельно новый признак - год основания стартапа.

In [ ]:
df_train['foundation_year'] = df_train['founded_at'].dt.year
df_test['foundation_year']  = df_test['founded_at'].dt.year

df_train.shape, df_test.shape

Добавим бинарный признак - был ли стартап основан до своего первого раунда финансирования или после.

In [ ]:
df_train['funded_before_founding'] = df_train['founded_at'] > df_train['first_funding_at']
df_test['funded_before_founding']  = df_test['founded_at']  > df_test['first_funding_at']

df_train.shape, df_test.shape

In [ ]:
df_train[df_train['funded_before_founding']].shape

Также, для целостности картины, добавим и признак "время между созданием стартапа и первым раундом финансирования". Такой признак скорее всего будет сильно коррелировать с новым признаком funded_before_founding, но funded_before_founding - бинарный признак, а создаваемый - дискретный. Потенциально такой признак может содержать больше информации в себе, но это проверим на этапе корреляционного анализа.

In [ ]:
df_train['first_funding_after_foundation'] = df_train['first_funding_at'] - df_train['founded_at']
df_test['first_funding_after_foundation']  = df_test['first_funding_at']  - df_test['founded_at']

df_train['first_funding_after_foundation'] = df_train['first_funding_after_foundation'].dt.days
df_test['first_funding_after_foundation']  = df_test['first_funding_after_foundation'].dt.days

df_train.shape, df_test.shape

In [ ]:
df_train.head(3)

In [ ]:
sns.relplot(data=df_train, x="foundation_year", y="first_funding_after_foundation")
plt.show()

In [ ]:
df_train[df_train['first_funding_after_foundation'] < 0].shape

Следующий признак предположительно будет коррелировать с признаком **funding_rounds**, поэтому пока добавим его, а на этапе корреляционного анализа решим, оставить его или удалить.

In [ ]:
df_train['first_last_funding_gap'] = df_train['last_funding_at'] - df_train['first_funding_at']
df_train['first_last_funding_gap'] = df_train['first_last_funding_gap'].dt.days

df_test['first_last_funding_gap']  = df_test['last_funding_at']  - df_test['first_funding_at']
df_test['first_last_funding_gap']  = df_test['first_last_funding_gap'].dt.days

df_train.shape, df_test.shape

In [ ]:
df_train.head(3)

---

### region, city, state_code

In [ ]:
# Можно добавить бинарный признак с информацией о городе, 
# в котором был создан стартапа, если это крупный город, 
# вроде Нью-Йорка или Лондона. Аналогично можно сделать что-то с самыми успешными штатами.

Удалим малоинформативные и слишком разнообразные кат. признаки "регион" и "город".

In [ ]:
df_train = df_train.drop(columns=['region', 'city', 'state_code'])
df_test  = df_test.drop(columns=['region', 'city', 'state_code'])

df_train.shape, df_test.shape

Добавим новые признаки - континент стартапа и субрегион.

In [ ]:
url = 'https://raw.githubusercontent.com/Lighter01/startups_categories/main/countryContinent.csv'

local_file_path = './datasets/countryContinent.csv'

if os.path.exists(local_file_path):
    print("File already exists.")
else:
    response = requests.get(url)
    
    if response.status_code == 200:
        with open('./datasets/countryContinent.csv', 'wb') as csv_file:
            csv_file.write(response.content)
        print("CSV file downloaded successfully.")
    else:
        print("Failed to download CSV file.")

In [ ]:
continents = pd.read_csv('./datasets/countryContinent.csv', sep=',', encoding='cp1252').drop(columns=['country_code'])
continents = continents.rename(columns={"code_3": "country_code"})
continents.loc[continents['country'] == 'Romania', 'country_code'] = 'ROM'
continents.loc[continents['country'] == 'Bahamas', 'country_code'] = 'BAH'
continents = continents[['country_code', 'continent', 'sub_region']]
continents.loc[continents.shape[0]] = ['TAN', 'Africa', 'Eastern Africa']

df_train = df_train.join(continents.set_index('country_code'), on='country_code')
display(df_train.head())

In [ ]:
display(df_train[(df_train['country_code'].notna()) & (df_train['continent'].isna())])

In [ ]:
df_train['country_code'].isna().sum(), df_train['continent'].isna().sum(), df_train['sub_region'].isna().sum()

In [ ]:
df_train['continent'].value_counts(), df_train['sub_region'].value_counts()

In [ ]:
df_test = df_test.join(continents.set_index('country_code'), on='country_code')
display(df_test.head())

In [ ]:
display(df_test[(df_test['country_code'].notna()) & (df_test['continent'].isna())])

In [ ]:
df_test['country_code'].isna().sum(), df_test['continent'].isna().sum(), df_test['sub_region'].isna().sum()

In [ ]:
df_test['continent'].value_counts(), df_test['sub_region'].value_counts()

---

### Даты

Удалим все признаки, связанные с датами. Во-первых, признак **founded_at** был упрощен до года создания стартапа. Во-вторых, **first_funding_at** и **last_funding_at** были преобразованы в новые признаки - кол-во дней между созданием стартапа и первым финансированием и бинарный признак, был ли стартап создан до или после своего первого финансирования. **closed_at** вообще никак нельзя сохранять, так как это признак, напрямую связанный с целевым параметром, а стоящая задача заключается в том, чтобы предсказывать состояние стартапа в будущем.

In [ ]:
df_train = df_train.drop(columns=['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])
df_test  = df_test.drop(columns=['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])

df_train.shape, df_test.shape

---

# Корреляционный анализ и отбор признаков

In [ ]:
df_train.dtypes

In [ ]:
def plot_cor_mat(correlation_matrix, mask=True, colormap='coolwarm', params={}):
    plt.figure(figsize=(10, 8))

    if mask:
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap=colormap, fmt='.2f', cbar_kws={"shrink": 0.75}, annot_kws={"fontsize": "small"}, **params)
    else:
        sns.heatmap(correlation_matrix, annot=True, cmap=colormap, fmt='.2f', cbar_kws={"shrink": 0.75}, annot_kws={"fontsize": "small"})
        
    plt.title('Phi-K correlation matrix')
    plt.tick_params(axis='x', labelsize=12)
    plt.tick_params(axis='y', labelsize=12)
    plt.show()

In [ ]:
#phik
interval_cols = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
print('Interval columns: ', interval_cols)

correlation_matrix = df_train.phik_matrix(interval_cols=interval_cols, dropna=True)

In [ ]:
plot_cor_mat(correlation_matrix)

- Из географических признаков оставим один. Пусть это будет параметр **continent**. Хоть номинально country_code коррелирует сильнее с целевой переменной, но это достигается за счет большего числа уникальных значений данного признака. Т.е. размеры выборки по каждой стране, во-первых, меньше, а во-вторых, их размеры распределены неравномерно. Группировка стран по географическому признаку создаст более крупные обобщенные группы, увеличит подвыборки, самих групп будет меньше, но при этом они всё еще будут содержать информацию о своих странах. sub_region выбирать не будем, т.к. sub_region разбивает записи на немного большее число групп, что приводит к образованию классов, в которые входят всего одна-две страны.
- Признак **mean_funding_sum** оказался неудачным, т.к. почти совсем не помогает разделить целевой показатель на два класса. Кроме того он, очевидно, коррелирует с **funding_total_usd**, что скорее всего связано с тем, что в выборке находится очень много записей о стартапах, у которых был всего один раунд финансирования. Корреляции с целевым параметром у обоих признаков не обнаружено. Поэтому удалим эти два признака, оставив последний признак, содержащий информацию о доходах стартапов - **log_funding_total**. Данный параметр имеет слабую корреляцию с целевой переменной и имеет среднюю информационную ценность по mutual_info и ANOVA.
- Признаки **first_funding_after_foundation** и **foundation_year** имеют сильную линейную связь, при этом каждый из признаков по отдельности довольно информативен, хоть и имеет слабую корреляцию с целевой переменной. Оставим все же first_funding_after_foundation, т.к. хоть корреляция более сильная у foundation_year, но у параметра года основания очень плохая интерпретируемость, т.е. сам по себе параметр позволит скорее идентифицировать более успешные года в целом, что позволит придерживаться логики, что в среднем если в году было больше удачных стартапов, то и одного конкретного шансы выше. first_funding_after_foundation в этом плане является более универсальным параметром, описывающим скорее скорее поведенческую черту стартапа, нежели какую-то статическую характеристику (очень запутанно, извините).
- category_group и main_category имеют идеальную корреляцию, т.к. один признак был порожден другим, т.е. имеется однозначеное отображение из одного пространства признаков в другое. В данном случае мне сложно выбрать что-то одно. Поэтому выберу один из признаков с расчетом, что в случае необходимости можно будет переобучить модель на другой подвыборке признаков. На  данный момент оставим признак **category_group**, т.к. он является более компактным признаком, включающим значительно меньше классов, нежели признак main_category.
- Признаки **funding_rounds**, **first_last_funding_gap** и **multidisciplinary** оставим.

In [ ]:
significance_overview = df_train.significance_matrix(interval_cols=interval_cols)

In [ ]:
plot_cor_mat(significance_overview, colormap=sns.color_palette("light:#5A9", as_cmap=True), params={"vmin":-5, "vmax":5})

In [ ]:
df_tmp = df_train.dropna()

X, y = df_tmp.drop(columns=['status']), df_tmp['status']
discrete_features = ((X.dtypes == 'int64') | (X.dtypes == int))
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

In [ ]:
# selector = SelectKBest(lambda x, y: mutual_info_classif(x, y, discrete_features=discrete_features) , k=1)
# X_reduced = selector.fit_transform(X, y)

# cols = selector.get_support(indices=True)
# selected_columns = X.iloc[:,cols].columns.tolist()
# selected_columns

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[:3]

In [ ]:
def plot_scores(scores, title=''):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title(title)


plt.figure(figsize=(8, 5))
plot_scores(mi_scores, title="Mutual Information Scores")

In [ ]:
def make_f_scores(X, y):
    F_scores, pvalues = f_classif(X, y)
    F_scores = pd.Series(F_scores, name="F Scores", index=X.columns)
    F_scores = F_scores.sort_values(ascending=False)
    return F_scores, pvalues

F_scores, pvalues = make_f_scores(X, y)
F_scores[:5]

In [ ]:
plot_scores(F_scores, title="F Scores")
display(pd.DataFrame(data={'F Scores': F_scores, 'p values': pvalues}).T)

Удаляем все ненужные признаки.

In [ ]:
df_train_final = df_train.drop(columns=['funding_total_usd', 'country_code', 'sub_region',
                                        'mean_funding_sum', 'foundation_year', 'main_category'])
df_test_final  = df_test.drop(columns =['funding_total_usd', 'country_code', 'sub_region',
                                        'mean_funding_sum', 'foundation_year', 'main_category'])

df_train_final.shape, df_test_final.shape

In [ ]:
fig = px.scatter_matrix(df_train_final, dimensions=df_train_final.select_dtypes(include=[int, float]).columns, color='status')

fig.update_layout(
    title='',
    width=1050, 
    height=1050,
)

fig.show()

---

# Обработка пропусков

Преобразуем все категориальные признаки к типу "category", потому что модели, которые будут использованы позже, предпочитают работать с этим типом категориальной переменной.
Также заменим в категориальных столбцах пустые строки на nan.

Для заполнения пропусков воспользуемся библиотекой miceforest, реализующей алгоритм заполнения пропусков MICE. В качестве базовой модели, используемой для промежуточных предсказаний на итерациях, используется LightGBM.

In [ ]:
cat_cols = df_train_final.select_dtypes(include=['object']).columns
df_train_final[cat_cols] = df_train_final[cat_cols].astype('category')
df_train_final = df_train_final.replace('', np.nan)
df_train_final.dtypes 

In [ ]:
cat_cols = df_test_final.select_dtypes(include=['object']).columns
df_test_final[cat_cols] = df_test_final[cat_cols].astype('category')
df_test_final = df_test_final.replace('', np.nan)
df_test_final.dtypes

In [ ]:
display_nans(df_train_final)
display_nans(df_test_final)

In [ ]:
def plot_scatterplots(df, target=''):
    val_cols = df.select_dtypes(include=[int, float]).columns
    # cat_cols = df.select_dtypes(include=['object']).columns
    tar_col = df[target]
    
    for val_col in val_cols:
        scatter_fig = px.scatter(df, x=val_col, y=tar_col, trendline="ols", trendline_color_override="red")
        scatter_fig.update_layout(title=f'')
        
        scatter_fig.show()

In [ ]:
plot_scatterplots(df_train_final, 'log_funding_total_usd')

In [ ]:
def imputer(X):
    scheme_mmc = mean_match_default.copy()
    scheme_mmc.set_mean_match_candidates(0) # замена 0 на любое другое число приводит к ошибке.
    
    # categorical_features = X.select_dtypes(include=['category']).columns.to_list()
    
    kernel = mf.ImputationKernel(
        X,
        mean_match_scheme = scheme_mmc, 
        datasets = 1,
        # categorical_feature = categorical_features,
        random_state=RANDOM_STATE
    )    

    variable_parameters = {
      'log_funding_total_usd': {
          "objective": "regression",
          "metric": "rmse"
      },
      'continent': {
          'objective': 'multiclass',
          'metric': 'multi_logloss' # multi_error
      },
      'category_group': {
          'objective': 'multiclass',
          'metric': 'multi_logloss' # multi_error
      }
    }
        
    kernel.mice(
        iterations = 5,
        num_boost_round = 125,
        max_bin = 32,
        boosting = 'gbdt',
        variable_parameters = variable_parameters,
        min_data_in_leaf=10
    )

    return kernel.complete_data(dataset=0)

In [ ]:
X_tmp = df_train_final.drop(columns=['status'])

In [ ]:
print(X_tmp.isna().sum())

In [ ]:
X_tmp_imp = imputer(X_tmp)
X_tmp_imp.isna().sum()

In [ ]:
df_train_final_imputed = X_tmp_imp
del X_tmp_imp
df_train_final_imputed.head()

In [ ]:
df_train_final_imputed['status'] = df_train_final['status']
df_train_final = df_train_final_imputed
del df_train_final_imputed

df_train_final.shape

In [ ]:
print(df_test_final.isna().sum())

In [ ]:
df_test_final = imputer(df_test_final)
df_test_final.isna().sum()

In [ ]:
df_train_final[['funding_rounds', 'first_funding_after_foundation', 'first_last_funding_gap']] = \
    df_train_final[['funding_rounds', 'first_funding_after_foundation', 'first_last_funding_gap']].astype('int64')

df_train_final[['category_group', 'continent']] = df_train_final[['category_group', 'continent']].astype('category')

df_train_final['log_funding_total_usd'] = df_train_final['log_funding_total_usd'].astype('float64')

df_train_final[['multidisciplinary', 'funded_before_founding']] = df_train_final[['multidisciplinary', 'funded_before_founding']].astype(bool)

df_train_final.dtypes

In [ ]:
df_test_final[['funding_rounds', 'first_funding_after_foundation', 'first_last_funding_gap']] = \
    df_test_final[['funding_rounds', 'first_funding_after_foundation', 'first_last_funding_gap']].astype('int64')

df_test_final[['category_group', 'continent']] = df_test_final[['category_group', 'continent']].astype('category')

df_test_final['log_funding_total_usd'] = df_test_final['log_funding_total_usd'].astype('float64')

df_test_final[['multidisciplinary', 'funded_before_founding']] = df_test_final[['multidisciplinary', 'funded_before_founding']].astype(bool)

df_test_final.dtypes

In [ ]:
display_nans(df_train_final)
display_nans(df_test_final)

---

# Подготовка моделей

В рамках данной работы упор будет сделан на модели градиентного бустинга на деревьях решений. Вначале обучим простой случайный лес, после чего перейдем к двум современным представителям градиентных моделей - XGBoost и LightGBM. Обучаться модели будут с применением Optuna - фреймворка для оптимизации гиперпараметров, использующего в своей основе байесовскую оптимизацию. В конце выберем одну модель, которая продемонстрирует самые высокие показатели целевой метрики.

In [ ]:
plot_size(df_train_final, 'status', labels='', explode=(0, 0.05), palette=sns.color_palette('Set2'))

In [ ]:
X, y = df_train_final.drop(columns=['status']), df_train_final['status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, shuffle=True, stratify=y, random_state=RANDOM_STATE)
X_train.shape, X_test.shape

In [ ]:
# upsampling = SMOTE(random_state=RANDOM_STATE)
# X_train_smote, y_train_smote = upsampling.fit_resample(X_train, y_train)
# X_train_smote.shape, y_train_smote.shape

In [ ]:
X.dtypes

In [ ]:
categorical_features = X.select_dtypes(include=['category']).columns.tolist()
numerical_features   = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features, numerical_features

In [ ]:
# Pipeline с применением SMOTENC, не требует кодировки кат. признаков перед применением апсемплера

transformer = ColumnTransformer(
                transformers = [
                    ('std_scaler', StandardScaler(), numerical_features[1:]),
                    ('target_encoder', TargetEncoder(random_state=RANDOM_STATE, target_type='binary'), ['category_group']),
                    ('ohe', OneHotEncoder(drop='first', sparse_output=False), ['multidisciplinary', 'funded_before_founding', 'continent']),
                ],
                remainder='passthrough'
            ).set_output(transform='pandas')

smotenc = SMOTENC(categorical_features=categorical_features,
                  random_state=RANDOM_STATE, 
                  sampling_strategy=0.75)
rand_downsampler = RandomUnderSampler(sampling_strategy=0.8, random_state=RANDOM_STATE)

preprocessor_nc = imbpipeline(
                    steps = [
                        ('umsampling', smotenc),
                        ('downsampling', rand_downsampler),
                        ('transforming', transformer)
                    ]
                   )

In [ ]:
def rf_objective_fn(trial, X, y):
    start_time = time.time()
    
    params = {
        "n_estimators":      trial.suggest_int('n_estimators', 128, 512, step=32),
        "max_depth":         trial.suggest_int('max_depth', 3, 8),
        "criterion":         trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss']),
        "max_features":      trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 150),
        "min_samples_leaf":  trial.suggest_int('min_samples_leaf', 1, 60),
        "bootstrap":         trial.suggest_categorical('bootstrap', [True, False])
    }

    if params['bootstrap']:
        params['max_samples'] = trial.suggest_float('max_samples', 0.6, 1, step=0.1)
    
    model = RandomForestClassifier(random_state=862, n_jobs=3, **params)

    model_pipeline = imbpipeline(
                        steps = [
                            ('umsampling', smotenc),
                            ('downsampling', rand_downsampler),
                            ('transforming', transformer),
                            ('fitting', model)
                        ]
                       )
    
    cross_val = StratifiedKFold(n_splits=5, shuffle=True, random_state=862)
    scoring = {
        'f1_score': make_scorer(f1_score, average='binary', pos_label='closed'),
    }
    cv_scores = cross_validate(model_pipeline, X, y, n_jobs=5, cv=cross_val, scoring=scoring, error_score='raise')
    
    scores = []
    for metric in scoring.keys():
        scores.append(np.mean(cv_scores['test_' + metric]))
    
    end_time = time.time()
    print(f"Trial {trial.number} elapsed time:", end_time - start_time, "seconds")
    
    return scores

In [ ]:
rf_study = optuna.create_study(directions=["maximize"], sampler=optuna.samplers.TPESampler(seed=862))
rf_study.optimize(lambda trial: rf_objective_fn(trial, X_train, y_train), n_trials=5)

In [ ]:
def xgb_objective_fn(trial, X, y):
    start_time = time.time()

    # dX_matrix = xgb.DMatrix(X, y)
    
    params = {
        "n_estimators":       trial.suggest_int('n_estimators', 128, 512, step=32),
        "max_depth":          trial.suggest_int('max_depth', 3, 8),
        "learning_rate":      trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        "subsample":          trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        "colsample_bytree":   trial.suggest_float('colsample_bytree', 0.4, 1),
        # "lambda":            trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # "alpha":             trial.suggest_float("alpha", 1e-8, 1.0, log=True)
        "lambda":             trial.suggest_float("lambda", 1e-3, 10.0, log=True),
        "alpha":              trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        # "enable_categorical": True # Мусор без DMatrix
    }

    model = xgb.XGBClassifier(objective='binary:logistic',
                              device='cuda',
                              tree_method='hist',
                              seed=862,
                              **params
    )

    model_pipeline = imbpipeline(
                        steps = [
                            ('umsampling', smotenc),
                            ('downsampling', rand_downsampler),
                            ('transforming', transformer),
                            ('fitting', model)
                        ]
                       )
    
    cross_val = StratifiedKFold(n_splits=5, shuffle=True, random_state=862)
    scoring = {
        'f1_score': make_scorer(f1_score, average='binary', pos_label=1),
    }
    cv_scores = cross_validate(model_pipeline, 
                               X, 
                               [int(stat == 'closed') for stat in y], 
                               n_jobs=5, cv=cross_val, scoring=scoring, error_score='raise')
    
    scores = []
    for metric in scoring.keys():
        scores.append(np.mean(cv_scores['test_' + metric]))
    
    end_time = time.time()
    print(f"Trial {trial.number} elapsed time:", end_time - start_time, "seconds")
    
    return scores

In [ ]:
xgb_study = optuna.create_study(directions=["maximize"], sampler=optuna.samplers.TPESampler(seed=862))
xgb_study.optimize(lambda trial: xgb_objective_fn(trial, X_train, y_train), n_trials=5)

---

In [ ]:
def lgb_objective_fn(trial, X, y):
    start_time = time.time()
    
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "max_bin": 32,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "num_boost_round": trial.suggest_int("num_boost_round", 100, 500),
    }

    model = lgb.LGBMClassifier(**params, device='cpu')

    model_pipeline = imbpipeline(
                        steps = [
                            ('umsampling', smotenc),
                            ('downsampling', rand_downsampler),
                            ('transforming', transformer),
                            ('fitting', model)
                        ]
                       )
    
    cross_val = StratifiedKFold(n_splits=5, shuffle=True, random_state=862)
    scoring = {
        'f1_score': make_scorer(f1_score, average='binary', pos_label=1),
    }
    cv_scores = cross_validate(model_pipeline, 
                               X, 
                               [int(stat == 'closed') for stat in y], 
                               n_jobs=5, cv=cross_val, scoring=scoring, error_score='raise')
    
    scores = []
    for metric in scoring.keys():
        scores.append(np.mean(cv_scores['test_' + metric]))
    
    end_time = time.time()
    print(f"Trial {trial.number} elapsed time:", end_time - start_time, "seconds")
    
    return scores

In [ ]:
lgb_study = optuna.create_study(directions=["maximize"], sampler=optuna.samplers.TPESampler(seed=862))
lgb_study.optimize(lambda trial: lgb_objective_fn(trial, X_train, y_train), n_trials=5)

---

# Анализ лучшей модели

Итак, лучше всего себя показала модель классификации XGBoost. Теперь рассмотрим данную модель подробнее, проанализируем важность признаков для нее,

In [ ]:
xgb_clf = xgb.XGBClassifier(**xgb_study.best_params)

In [ ]:
model_pipeline = imbpipeline(
                        steps = [
                            ('umsampling', smotenc),
                            ('downsampling', rand_downsampler),
                            ('transforming', transformer),
                            ('fitting', xgb_clf)
                        ]
                       )
model_pipeline.fit(X_train, [1 if status=='closed' else 0 for status in y_train])

In [ ]:
df_show = pd.DataFrame(xgb_clf.feature_importances_, columns=['Feature Importance'], index=xgb_clf.feature_names_in_).sort_values(by='Feature Importance', ascending=False)
display(df_show.style.background_gradient(cmap='Blues'))

fig, ax = plt.subplots(1, 1, figsize=(8, 5))

y_pos = np.arange(len(df_show))

ax.barh(df_show.index[::-1], df_show['Feature Importance'][::-1], alpha=0.8, edgecolor='black', linewidth=1)
ax.set_xlabel('Feature Importance', fontsize=14)
ax.set_title('Feature Importance for each column', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=11)
ax.grid(axis='x', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
xgb_clf = model_pipeline.named_steps['fitting']
umsampler_xgb   = model_pipeline.named_steps['umsampling']
downsample_xgb  = model_pipeline.named_steps['downsampling']
transformer_xgb = model_pipeline.named_steps['transforming']

In [ ]:
def test_classification(model, X, y, title=''):
    y_pred = model.predict(transformer_xgb.transform(X))
    y_pred = ['closed' if status == 1 else 'operating' for status in y_pred]

    print(f'Accuracy: {accuracy_score(y, y_pred)}')
    print(f'Precision: {precision_score(y, y_pred, pos_label='closed')}')
    print(f'Recall: {recall_score(y, y_pred, pos_label='closed')}')
    print(f'F1: {f1_score(y, y_pred, pos_label='closed')}')

    conf_matrix = confusion_matrix(y, y_pred)
    ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['closed', 'operating']).plot()
    plt.title(title)
    plt.show()

In [ ]:
test_classification(xgb_clf, X_test, y_test)

In [ ]:
data = transformer_xgb.transform(downsample_xgb.fit_resample(*umsampler_xgb.fit_resample(X_test, y_test))[0])
data

In [ ]:
sample_ind = data.reset_index().sample(n=75, random_state=RANDOM_STATE).index
sample_ind

In [ ]:
explainer = shap.TreeExplainer(xgb_clf)
shap_values = explainer(data)

In [ ]:
shap.initjs()

In [ ]:
shap.plots.force(explainer.expected_value, shap_values.values[sample_ind, :], feature_names=data.columns)

In [ ]:
shap.plots.force(explainer.expected_value, shap_values.values[862, :], feature_names=data.columns)

In [ ]:
shap.plots.force(explainer.expected_value, shap_values.values[850, :], feature_names=data.columns)

In [ ]:
shap.plots.waterfall(shap_values[862, :], max_display=14)

In [ ]:
shap.plots.waterfall(shap_values[850, :], max_display=14)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.plots.beeswarm(shap_values) # summary_plot

Лучшей моделью среди рассмотренных оказался модель градиентного бустинга на решающими деревьями - XGBoost. Итоговая модель имеет далеко не самые выдающиеся результаты - всего 0.227 по F1.

Анализ важности признаков модели позваоляет сделать следующие наблюдения:

- самыми влиятельными признаками оказались: географическое расположение стартапа (континент), бинарный признак "multidisciplinary", который выделял те стартапы, которые обладали несколькими категориями, а также категориальная группа стартапа и количество раундов финанисрования;
- вероятно, отдельные категории стартапов действительно дают модели больше уверенности в исходе судьбы стартапа, однако также значительная часть классов (групп) едва ли возволяет провести какую-либо дихотомию; 
- незначительно на предсказания влияют: сумма финансирования стартапа (прологарифмированная), а также бинарный признак, описывающий последовательность финансирования/открытия стартапа;
- стартапы из Америки действительно являются более успешыми, а стартапы из Европы и Азии - наоборот - чаще проваливаются и закрываются;
- время между первым и последним раундом финансирования тоже играют определенную роль. Увеличение временного зазора между двумя событиямим влияет на вероятность модели предсказать закрытие стартапа;
- время между открытием стартапа и первыми инвестициями в него тоже играет не последнюю роль для модели. Можно заметить, что меньший срок ожидания первых инвестиций способствует большей уверенности в успех бизнеса.

---

# Итоговые предсказания

In [ ]:
predictions = xgb_clf.predict(transformer_xgb.transform(df_test_final))

In [ ]:
submit = pd.DataFrame({'name': df_sampsub.name.values, 'status': predictions})

In [ ]:
submit

In [ ]:
submit.to_csv('./datasets/submission.csv', index=False)

---